<a href="https://colab.research.google.com/github/anadiedrichs/2023-vision-computacional/blob/main/TP3_vision_Diedrichs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Trabajo Práctico Nro 3


Ana Laura Diedrichs

SIU a0902

9Co2022


 Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en Material_TPs/TP3/images a partir del template Material_TPs/TP3/template




In [ ]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
%matplotlib inline
# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV
import cv2 as cv
# matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt
# para importar imagenes desde urls
from skimage import io
# mostrar imagenes opencv en googlecolab
from google.colab.patches import cv2_imshow

Muestro la imagen del template

In [ ]:
url_template = "https://www.dropbox.com/s/rn5d4oubths7xmz/pattern.png?dl=1"
img = io.imread(url_template)
img_template = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

plt.figure(figsize=(8,8))
plt.imshow(img_template, cmap='gray')

In [ ]:
img_nivel_1 = cv.pyrDown(img_template)
plt.figure(figsize=(8,8))
plt.imshow(img_nivel_1, cmap='gray')

img_template = img_nivel_1

## 1.(4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos


In [ ]:
urls = ["https://www.dropbox.com/s/lxv9drjd0vzmzg2/COCA-COLA-LOGO.jpg?dl=1",
     "https://www.dropbox.com/s/bifxxgy5l0vicf0/coca_logo_1.png?dl=1",
      "https://www.dropbox.com/s/a4y0lsjykqzmm7b/coca_logo_2.png?dl=1",
      "https://www.dropbox.com/s/o1rtc9zsu81u0eq/coca_retro_1.png?dl=1",
      "https://www.dropbox.com/s/jgxf1g1v94wdw6r/coca_retro_2.png?dl=1",
      "https://www.dropbox.com/s/pfwbo9nnyi5ifyq/logo_1.png?dl=1",
      "https://www.dropbox.com/s/z0sfx2k08ou61at/coca_multi.png?dl=1"]

w = img_template.shape[1]
h = img_template.shape[0]

# Los 6 métodos posibles para comparación:
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']

for url in urls: # por cada imagen
  myImg = io.imread(url)
  img_rgb= cv.cvtColor(myImg, cv.COLOR_BGR2RGB)
  img_gray= cv.cvtColor(myImg, cv.COLOR_BGR2GRAY)

  for meth in methods: # por cada uno de los métodos
    # Hago una copia de la imagen porque ciclo a ciclo le dibujo rectángulos
    img_salida = img_rgb.copy()

    method = eval(meth)

    # Aplicamos la coincidencia de patrones
    #--------------------------------------
    res = cv.matchTemplate(img_gray, img_template, method)

    # Encontramos los valores máximos y mínimos
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)

    # Si el método es TM_SQDIFF o TM_SQDIFF_NORMED, tomamos el mínimo
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc

    # Marcamos el lugar donde lo haya encontrado
    #----------------------------------------
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv.rectangle(img_salida,top_left, bottom_right, 255, 2)

    # Graficamos el procesamiento y la salida
    #----------------------------------------
    plt.figure()

    # Resultado de coincidencia
    plt.subplot(121),plt.imshow(res,cmap = 'gray')
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])

    # Imagen original con recuadros
    plt.subplot(122),plt.imshow(img_salida)
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])

    plt.suptitle(meth)
    plt.show()



In [ ]:
template_image = img_template

height, width = template_image.shape

for url in urls: # por cada imagen
  myImg = io.imread(url)
  img_rgb= cv.cvtColor(myImg, cv.COLOR_BGR2RGB)
  target_image= cv.cvtColor(myImg, cv.COLOR_BGR2GRAY)

  # Realizar el template matching
  result = cv.matchTemplate(target_image, template_image, cv.TM_CCOEFF_NORMED)

  # Establecer un umbral para la detección
  threshold = 0.70
  loc = np.where(result >= threshold)

  img_salida = img_rgb.copy()

  for pt in zip(*loc[::-1]):
    cv.rectangle(img_salida, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)

  # Graficamos el procesamiento y la salida
  #----------------------------------------
  plt.figure()

  # Resultado de coincidencia
  plt.subplot(121),plt.imshow(res,cmap = 'gray')
  plt.title('Matching Result'), plt.xticks([]), plt.yticks([])

  # Imagen original con recuadros
  plt.subplot(122),plt.imshow(img_salida)
  plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
  plt.show()




##2.(4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen
coca_multi.png con el mismo template del ítem 1


##3.(2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.
Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza
de la detección.
